In [1]:
from gascooling.accommodation.evaluation import experiment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import odr
from scipy.interpolate import splev, splrep

In [2]:
TEMP_FIELD = "Temperature_Testmass_(K)"
PD_FIELD = "Voltage_Photodiode_(V)"
MEASUREMENT_NAME = "live"

base_connector = experiment.baserow.Connector()
PD_CALIBRATIONS = base_connector.pd_calibration_at_low_pressure()


def f(B, x):
    """Linear function y = m*x + b"""
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0] * x + B[1]


calibration_collection = experiment.calibration.CalibrationCollection()
cali = experiment.calibration.LowPressureCalibration()
_, data = cali._fit_photo_voltage_to_discommoding_cooling_power

In [3]:
cooldowns_evals = cali.cooldowns
print(data)
fig, ax = plt.subplots()

# print(data)

ax.errorbar(
    y=[power * 1000 for power in data["laser_power"]],
    x=[photo_voltage * 1000 for photo_voltage in data.index],
    yerr=[power * 1000 for power in data["laser_power_err"]],
    fmt=",",
    zorder=1,
)

In [11]:
fig1, ax1 = plt.subplots()
ax1.errorbar(
    x=temperatures,
    y=[photo_voltage * 1000 for photo_voltage in photo_voltages],
    yerr=[photo_voltage * 1000 for photo_voltage in photo_voltages_err],
    fmt="o",
)
ax1.set_xlabel("Equilibrated TM temperature without gas (K)")
ax1.set_ylabel("Photodiode Voltage (mV)")
plt.tight_layout()
plt.grid(visible=True, which="both", axis="both")

# Fit
linear = odr.Model(f)
fit_data = odr.RealData(
    x=temperatures[:13], y=photo_voltages[:13]
)  # , sx=laser_powers_err, sy=photo_voltages_err)
fit_model = odr.ODR(fit_data, linear, beta0=[-0.01, -0.0095])
fit_result = fit_model.run()

print(fit_result.beta)
# Plot fit
xs = np.arange(7, 100, step=0.5)
ys = f(fit_result.beta, xs)
ax1.plot(xs, [y * 1000 for y in ys])
plt.grid(visible=True, which="both", axis="both")

In [35]:
df = pd.DataFrame(
    data={
        "laser_power": laser_powers,
        "laser_power_err": laser_powers_err,
        "photo_voltage": photo_voltages,
    }
)
df = df.set_index("photo_voltage")
df = df.sort_index()
fig2, ax2 = plt.subplots()
ax2.errorbar(
    y=[power * 1000 for power in laser_powers],
    x=[photo_voltage * 1000 for photo_voltage in photo_voltages],
    yerr=[err * 1000 for err in laser_powers_err],
    xerr=[photo_voltage * 1000 for photo_voltage in photo_voltages_err],
    fmt=".",
    elinewidth=2,
    zorder=1,
)
spline = splrep(
    y=df["laser_power"],
    x=df.index,
    s=9e-5,
    # w=[1/err for err in df["laser_power_err"]]
)
interpolated_f = spline
xs = np.linspace(-0.8, 0, num=300)
# ys = interpolated_f(xs)
ys = splev(xs, spline)
ax2.plot([x * 1000 for x in xs], [y * 1000 for y in ys], zorder=2)
ax2.set_ylabel("Absorbed Laser power (mW)")
ax2.set_xlabel("Photodiode Voltage (mV)")
plt.tight_layout()
plt.grid(visible=True, which="both", axis="both")

In [13]:
fig3, ax3 = plt.subplots()
ax3.errorbar(
    x=[power * 1000 for power in laser_powers],
    y=temperatures,
    xerr=[err * 1000 for err in laser_powers_err],
    fmt="o",
)
ax3.set_xlabel("Discommoding cooling (mW)")
ax3.set_ylabel("Test mass temperature without gas (K)")
plt.tight_layout()
plt.grid(visible=True, which="both", axis="both")

In [7]:
fig4, ax4 = plt.subplots()
xs = np.linspace(10, 140, num=100)
ys = [calibration_collection.discommoding_cooling_by_temperature(x).nominal_value for x in xs]
ax4.plot(
    xs,
    ys,
)
ax4.set_ylabel("Interpolated discommoding cooling (mW)")
ax4.set_xlabel("Test mass temperature (K)")
plt.tight_layout()
plt.grid(visible=True, which="both", axis="both")